In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9600355742670163293
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4178575360
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1826886343502233441
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [2]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

        

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [4]:
#학습셋의 변형을 설정하는 부분입니다. 
train_datagen = ImageDataGenerator(rescale=1./255,          # 주어진 이미지의 크기를 설정합니다.
                                  #horizontal_flip=True,     # 수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  #width_shift_range=0.1,    # 전체 크기의 15% 범위에서 좌우로 이동합니다.
                                  #height_shift_range=0.1,   # 마찬가지로 위, 아래로 이동합니다.
                                  #rotation_range=5,        # 정해진 각도만큼 회전시킵니다.
                                  #shear_range=0.7,         # 좌표 하나를 고정시키고 나머지를 이동시킵니다.
                                  #zoom_range=1.2,          # 확대 또는 축소시킵니다.
                                  #vertical_flip=True,      # 수직 대칭 이미지를 만듭니다.
                                  #fill_mode='nearest'      # 빈 공간을 채우는 방법입니다. nearest 옵션은 가장 비슷한 색으로 채우게 됩니다.
                                  )  
train_generator = train_datagen.flow_from_directory('testC/clothes/train',
                                                   target_size=(220, 260),
                                                   batch_size=5,
                                                   class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('testC/clothes/test',
                                                   target_size=(220, 260),
                                                   batch_size=5,
                                                   class_mode='categorical')


Found 14000 images belonging to 7 classes.
Found 3506 images belonging to 7 classes.


In [5]:
transfer_model = VGG16(weights='imagenet', include_top = False,
                      input_shape=(220, 260,3))
transfer_model.trainable = False
transfer_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 220, 260, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 220, 260, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 220, 260, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 110, 130, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 110, 130, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 110, 130, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 55, 65, 128)       0     

In [6]:
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(layers.Flatten())
finetune_model.add(layers.Dense(64, activation = 'relu'))
finetune_model.add(layers.Dropout(0.5))
finetune_model.add(layers.Dense(7, activation = 'softmax'))
finetune_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 24576)             0         
                                                                 
 dense (Dense)               (None, 64)                1572928   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 7)                 455       
                                                                 
Total params: 16,288,071
Trainable params: 1,573,383
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
finetune_model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(learning_rate=0.0002),
             metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss',patience=5)


In [8]:
import PIL
from PIL import Image

In [9]:
history = finetune_model.fit(
       train_generator,
       epochs=20,
       validation_data=test_generator,
       callbacks=[early_stopping])

Epoch 1/20
2800/2800 [==============================] - 157s 55ms/step - loss: 1.1173 - accuracy: 0.5493 - val_loss: 0.7607 - val_accuracy: 0.7151
Epoch 2/20
2800/2800 [==============================] - 155s 56ms/step - loss: 0.8612 - accuracy: 0.6436 - val_loss: 0.6573 - val_accuracy: 0.7442
Epoch 3/20
2800/2800 [==============================] - 156s 56ms/step - loss: 0.7769 - accuracy: 0.6788 - val_loss: 0.5927 - val_accuracy: 0.7815
Epoch 4/20
2800/2800 [==============================] - 156s 56ms/step - loss: 0.7305 - accuracy: 0.6998 - val_loss: 0.5885 - val_accuracy: 0.7741
Epoch 5/20
2800/2800 [==============================] - 156s 56ms/step - loss: 0.6892 - accuracy: 0.7146 - val_loss: 0.5582 - val_accuracy: 0.7958
Epoch 6/20
2800/2800 [==============================] - 156s 56ms/step - loss: 0.6519 - accuracy: 0.7286 - val_loss: 0.5829 - val_accuracy: 0.7983
Epoch 7/20
2800/2800 [==============================] - 156s 56ms/step - loss: 0.6130 - accuracy: 0.7542 - val_loss: 0